In [1]:
import numpy as np
import pandas as pd
from sympy import *

In [2]:
# 8 different cols corresponds to f_{++-}, ..., f_{-+-};
target = np.array([[1, 1, -1, -1, 1, 1, -1, -1],
                   [1, 1, -1, -1, -1, -1, 1, 1],
                   [-1, 1, -1, 1, 1, -1, 1, -1]])

# cleaner input later

mean = np.array([2.795084971874737,
 2.800317370942086,
 2.792356968942187,
 2.8109163331554354,
 2.781936892167038,
 2.776749214459239,
 2.8200842118631844,
 2.7832338115939885])

cov = np.array([[ 0.10815616, -0.01290591, -0.01658358, -0.01911512, -0.01469069,
        -0.01202002, -0.01986286, -0.01297798],
       [-0.01290591,  0.10223085, -0.01302075, -0.01820604, -0.01536957,
        -0.01587493, -0.00927608, -0.01757757],
       [-0.01658358, -0.01302075,  0.11190446, -0.01635016, -0.01486774,
        -0.02029532, -0.01258839, -0.01819853],
       [-0.01911512, -0.01820604, -0.01635016,  0.10672009, -0.01819004,
        -0.0120778 , -0.01157835, -0.01120258],
       [-0.01469069, -0.01536957, -0.01486774, -0.01819004,  0.11301614,
        -0.01713021, -0.01452683, -0.01824106],
       [-0.01202002, -0.01587493, -0.02029532, -0.0120778 , -0.01713021,
         0.10669449, -0.01572034, -0.01357588],
       [-0.01986286, -0.00927608, -0.01258839, -0.01157835, -0.01452683,
        -0.01572034,  0.09758462, -0.01403176],
       [-0.01297798, -0.01757757, -0.01819853, -0.01120258, -0.01824106,
        -0.01357588, -0.01403176,  0.10580536]])

# rv = np.random.multivariate_normal(mean, cov).reshape((8, 1))
# print(rv) # sample from multivariate gaussian 

In [4]:
rv = np.random.multivariate_normal(mean, cov).reshape((8, 1))
print(rv.T, "\n") # sample from multivariate gaussian

scales = [1] # later use np.linspace(1, 25, num=10)

for scale in scales:

  print("scale              ", scale)

  scaled_rv = (scale * rv).astype(int)

  # print("scaled random sample", scaled_rv.T)

  x = symbols('x')
  expr = 0 # negative log likelihood expression

  for i in range(0, 8):
    factor = target[0][i] * target[1][i] # assume theta_C = 0, so sigma_C_i does not matter
    for j in range(0, scaled_rv[i][0]):
      subexpr = 1 / 8 * (1 + factor * x) # probability of observing column i
      expr += -log(subexpr)

  print("nll expression     ", expr)    

  dexpr = diff(expr) # derivative of negative log likelihood expression
  solution = list(solveset(dexpr, x, Interval(0, 1).closure))
  nll = -1
  print("critical (T_A)(T_B)", solution)
  if len(solution) == 0: # if there exist no critical values, check 0 and 1
    min = 0
    max = 1
    nll_0 = expr.evalf(subs={x: min})
    nll_1 = expr.evalf(subs={x: max})
    if nll_0 <= nll_1:
      solution.append(min)
      nll = nll_0
    else:
      solution.append(max)
      nll = nll_1
  else: # plug in critical value to compute maximum likelihood
    nll = expr.evalf(subs={x: solution[0]})
    min = 0
    max = 1
    nll_0 = expr.evalf(subs={x: min})
    nll_1 = expr.evalf(subs={x: max})
    if nll_0 <= nll:
      solution.insert(0, min)
      nll = nll_0
    elif nll_1 <= nll:
      solution.insert(0, max)
      nll = nll_1
  lhd = exp(-1 * nll)
  print("optimal (T_A)(T_B) ", solution[0]) # (theta_A)(theta_B) maximizing likelihood
  print("maximum likelihood ", lhd, "\n") # maximum likelihood

[[2.77576394 2.62083737 2.75315434 2.71716411 3.01385441 2.7952545
  2.7727821  2.91190446]] 

scale               1
nll expression      -9*log(0.125 - 0.125*x) - 8*log(0.125*x + 0.125)
critical (T_A)(T_B) []
optimal (T_A)(T_B)  0
maximum likelihood  4.44089209850061e-16 

